In [1]:
import glob
import tifffile
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import keras_hub
from IPython import display
import tensorflow as tf
import os

2025-05-15 14:31:35.305786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747333895.317027   32942 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747333895.320410   32942 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-15 14:31:35.333665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/albert-wang/miniforge3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please u

In [2]:
mip_paths = sorted(glob.glob('../denoised-mips/*.tif'))
mip_paths

['../denoised-mips/021825_eft_41z_fed_worm001.tif',
 '../denoised-mips/022025_eft_41z_starved_worm002.tif',
 '../denoised-mips/022025_eft_41z_starved_worm003.tif',
 '../denoised-mips/022025_eft_41z_starved_worm004.tif',
 '../denoised-mips/022025_eft_41z_starved_worm005.tif',
 '../denoised-mips/022025_eft_41z_starved_worm006.tif',
 '../denoised-mips/022025_eft_41z_starved_worm007.tif',
 '../denoised-mips/042125_eft_fed_worm001.tif',
 '../denoised-mips/042125_eft_fed_worm002.tif',
 '../denoised-mips/042125_eft_fed_worm003.tif']

In [3]:
model = keras_hub.models.ImageSegmenter.from_preset('deeplab_v3_plus_resnet50_pascalvoc', num_classes=2, compile=False)
model.load_weights('deeplab_051525.weights.h5')

I0000 00:00:1747333896.917350   32942 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6192 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
def make_tensor(index):
    img = np.zeros((1, 512, 512, 3), np.float32)
    rfp_zscore = stats.zscore(mip[index, 1], axis=None)
    img[..., 0] = rfp_zscore
    img[..., 1] = rfp_zscore
    img[..., 2] = rfp_zscore
    tensor = tf.convert_to_tensor(img)
    return tensor

def make_label(tensor):
    label = model.predict(tensor)
    return np.argmax(label[0], axis=2)

In [5]:
for mip_path in mip_paths:
    name = os.path.splitext(os.path.basename(mip_path))[0]
    mip = tifffile.imread(mip_path)
    output = np.zeros((len(mip), 512, 512), bool)
    for i in range(len(output)):
        tensor = make_tensor(i)
        output[i] = make_label(tensor)
        display.clear_output(wait=True)
        print(f'{name}: {i}')
    tifffile.imwrite(f'../labels/{name}_label.tif', output)

042125_eft_fed_worm003: 700
